This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [49]:
# get api key from your .env file
import os
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')

#print(API_KEY)

Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [50]:
# First, import the relevant modules
import requests
import json

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [51]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
url = "https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json"
params = {
    "api_key": API_KEY,
    "start_date": "2023-11-01",
    "end_date": "2023-11-01"
}

response=requests.get(url, params=params)


In [52]:
#response.status_code

200

In [53]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure

if response.status_code == 200:
   
    data = response.json()
    for key in data.keys():
        print(f"Key: {key}")
    
else:
    
    print(f"Request error - Status Code: {response.status_code}")
    

Key: dataset_data


In [54]:
print(data["dataset_data"]["column_names"])

['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover']


In [55]:
if "dataset_data" in data:
    data_dict = data["dataset_data"]
   
    for key in data_dict.keys():
        print(f"Key within data_dict: {key}")
else:
    print("The 'dataset_data' key is not found in the JSON response.")

Key within data_dict: limit
Key within data_dict: transform
Key within data_dict: column_index
Key within data_dict: column_names
Key within data_dict: start_date
Key within data_dict: end_date
Key within data_dict: frequency
Key within data_dict: data
Key within data_dict: collapse
Key within data_dict: order


In [56]:
print(data['dataset_data']['column_names'])

['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover']


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [57]:
#Task 1: Collect data for AFX_X for the whole year 2017

start_date = "2017-01-01"
end_date = "2017-12-31"

response = requests.get(url, params={"api_key": API_KEY, "start_date": start_date, "end_date": end_date})

if response.status_code == 200:
    data = response.json()
else:
    print(f"Request error - Status Code: {response.status_code}")


In [62]:
# Task 2: Convert the JSON response to a Python dictionary
data_dict = data.get("dataset_data", {})

In [63]:
data_dict.keys()

dict_keys(['limit', 'transform', 'column_index', 'column_names', 'start_date', 'end_date', 'frequency', 'data', 'collapse', 'order'])

In [64]:
column_names = data_dict.get("column_names", [])
column_names

['Date',
 'Open',
 'High',
 'Low',
 'Close',
 'Change',
 'Traded Volume',
 'Turnover',
 'Last Price of the Day',
 'Daily Traded Units',
 'Daily Turnover']

In [65]:
# Task 3:Calculate what the highest and lowest opening prices were for the stock in this period.
opening_prices = []

data_list = data_dict.get("data", [])
open_price_index = column_names.index("Open")

highest_opening_price = None
lowest_opening_price = None

for entry in data_list:
    opening_price = entry[open_price_index]
    if opening_price is not None:
        if highest_opening_price is None or opening_price > highest_opening_price:
            highest_opening_price = opening_price
        if lowest_opening_price is None or opening_price < lowest_opening_price:
            lowest_opening_price = opening_price

print(f"Highest opening price in 2017: {highest_opening_price}")
print(f"Lowest opening price in 2017: {lowest_opening_price}")

Highest opening price in 2017: 53.11
Lowest opening price in 2017: 34.0


In [66]:
# Task4: What was the largest change in any one day (based on High and Low price)?

max_daily_change = None


high_price_index = column_names.index("High")
low_price_index = column_names.index("Low")

for entry in data_list:
    high_price = entry[high_price_index]
    low_price = entry[low_price_index]
    if high_price is not None and low_price is not None:
        daily_change = high_price - low_price
        if max_daily_change is None or daily_change > max_daily_change:
            max_daily_change = daily_change

print(f"Largest change in any one day in 2017: {max_daily_change}")



Largest change in any one day in 2017: 2.8100000000000023


In [67]:
# task 5: What was the largest change between any two days (based on Closing Price)?

max_2day_change = None


close_price_index = column_names.index("Close")

previous_day_close = None

for entry in data_list:
    close_price = entry[close_price_index]
    if close_price is not None:
        if previous_day_close is not None:
            two_day_change = abs(close_price - previous_day_close)
            if max_2day_change is None or two_day_change > max_2day_change:
                max_2day_change = two_day_change
        previous_day_close = close_price

print(f"Largest change between any 2  days in 2017 is: {max_2day_change}")


Largest change between any 2  days in 2017 is: 2.559999999999995


In [68]:
#Task 6 What was the average daily trading volume during this year?

total_volume = 0
total_days = 0

traded_volume_index = column_names.index("Traded Volume")

for entry in data_list:
    traded_volume = entry[traded_volume_index]
    if traded_volume is not None:
        total_volume += traded_volume
        total_days += 1

average_volume = total_volume / total_days

print(f"Average daily trading volume in 2017 is: {average_volume}")


Average daily trading volume in 2017 is: 89124.33725490196


In [69]:
#Task 7: What was the median trading volume during this year.

def cal_median(data):
    sorted_data = sorted(data)
    n = len(sorted_data)
    if n % 2 == 0:
        median = (sorted_data[n // 2 - 1] + sorted_data[n // 2]) / 2
    else:
        median = sorted_data[n // 2]
    return median

traded_volume_index = column_names.index("Traded Volume")

trading_volumes = []  # Corrected variable name

for entry in data_list:
    traded_volume = entry[traded_volume_index]
    if traded_volume is not None:
        trading_volumes.append(traded_volume)

median_volume = cal_median(trading_volumes)

print(f"Median trading volume in 2017 is : {median_volume}")


Median trading volume in 2017 is : 76286.0
